<a href="https://colab.research.google.com/github/Nilufayeasmin299/Reproduce-GNN_Ownership_Verification/blob/main/GNN_reproduction_with_different_settings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torch-geometric numpy scikit-learn matplotlib tqdm

In [ ]:
# Import required packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, GINConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import numpy as np
from tqdm import tqdm
import random
import os

In [ ]:
# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

In [ ]:

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class EnhancedGNN(nn.Module):
    """Enhanced GNN model with multiple architectures support"""
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2,
                 dropout=0.5, arch='GCN'):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.arch = arch

        # Initialize layers
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Input layer
        if arch == 'GCN':
            self.convs.append(GCNConv(in_channels, hidden_channels))
        elif arch == 'GAT':
            self.convs.append(GATConv(in_channels, hidden_channels))
        elif arch == 'SAGE':
            self.convs.append(SAGEConv(in_channels, hidden_channels))
        elif arch == 'GIN':
            gin_nn = nn.Sequential(
                nn.Linear(in_channels, hidden_channels),
                nn.ReLU(),
                nn.Linear(hidden_channels, hidden_channels)
            )
            self.convs.append(GINConv(gin_nn))

        self.batch_norms.append(nn.BatchNorm1d(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            if arch == 'GCN':
                self.convs.append(GCNConv(hidden_channels, hidden_channels))
            elif arch == 'GAT':
                self.convs.append(GATConv(hidden_channels, hidden_channels))
            elif arch == 'SAGE':
                self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            elif arch == 'GIN':
                gin_nn = nn.Sequential(
                    nn.Linear(hidden_channels, hidden_channels),
                    nn.ReLU(),
                    nn.Linear(hidden_channels, hidden_channels)
                )
                self.convs.append(GINConv(gin_nn))
            self.batch_norms.append(nn.BatchNorm1d(hidden_channels))

        # Output layer
        if arch == 'GCN':
            self.convs.append(GCNConv(hidden_channels, out_channels))
        elif arch == 'GAT':
            self.convs.append(GATConv(hidden_channels, out_channels))
        elif arch == 'SAGE':
            self.convs.append(SAGEConv(hidden_channels, out_channels))
        elif arch == 'GIN':
            gin_nn = nn.Sequential(
                nn.Linear(hidden_channels, hidden_channels),
                nn.ReLU(),
                nn.Linear(hidden_channels, out_channels)
            )
            self.convs.append(GINConv(gin_nn))

    def forward(self, x, edge_index):
        # Forward pass through layers
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index)
            x = self.batch_norms[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.convs[-1](x, edge_index)
        return x

In [ ]:
class VerificationClassifier(nn.Module):
    """Enhanced classifier for ownership verification"""
    def __init__(self, input_dim, hidden_dim=256):
        super().__init__()

        self.network = nn.Sequential(
            # First layer
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),

            # Second layer
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),

            # Third layer
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.BatchNorm1d(hidden_dim // 4),
            nn.ReLU(),
            nn.Dropout(0.3),

            # Output layer
            nn.Linear(hidden_dim // 4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)


In [ ]:
class GNNVerifier:
    """Enhanced GNN ownership verification system"""
    def __init__(self, hidden_dim=128, mask_ratio=0.1, lr=0.001, weight_decay=1e-4):
        self.hidden_dim = hidden_dim
        self.mask_ratio = mask_ratio
        self.lr = lr
        self.weight_decay = weight_decay
        self.device = device

    def create_masks(self, data):
        """Create feature masks with importance sampling"""
        num_features = data.x.size(1)

        # Calculate feature importance
        feature_importance = torch.abs(data.x).mean(0)
        importance_threshold = torch.quantile(feature_importance, self.mask_ratio)

        # Create mask based on importance
        mask = torch.ones_like(data.x)
        mask[:, feature_importance < importance_threshold] = 0

        # Apply mask
        data.x = data.x * mask
        return data

    def extract_fingerprint(self, model, data):
        """Extract enhanced model fingerprint"""
        model.eval()
        with torch.no_grad():
            # Forward pass
            outputs = model(data.x, data.edge_index)
            logits = outputs.detach()
            probs = F.softmax(logits, dim=1)

            # Calculate fingerprint components
            mean_logits = logits.mean(0)
            mean_probs = probs.mean(0)
            max_probs = probs.max(1)[0]
            entropy = -(probs * torch.log(probs + 1e-10)).sum(1).mean()

            # Layer activations
            layer_activations = []
            x = data.x
            for i, conv in enumerate(model.convs[:-1]):
                x = conv(x, data.edge_index)
                layer_activations.append(x.mean(0))

            # Combine all components
            fingerprint = torch.cat([
                mean_logits,
                mean_probs,
                max_probs.mean().unsqueeze(0),
                entropy.unsqueeze(0),
                *layer_activations
            ])

        return fingerprint

    def train_verifier(self, target_model, shadow_models, indep_models, train_data):
        """Train enhanced verification classifier"""
        # Extract fingerprints
        target_fp = self.extract_fingerprint(target_model, train_data)

        shadow_fps = []
        for model in tqdm(shadow_models, desc="Processing shadow models"):
            fp = self.extract_fingerprint(model, train_data)
            shadow_fps.append(fp)
        shadow_fps = torch.stack(shadow_fps)

        indep_fps = []
        for model in tqdm(indep_models, desc="Processing independent models"):
            fp = self.extract_fingerprint(model, train_data)
            indep_fps.append(fp)
        indep_fps = torch.stack(indep_fps)

        # Prepare training data
        X = torch.cat([shadow_fps, indep_fps], dim=0).to(device)
        y = torch.cat([
            torch.ones(len(shadow_fps)),
            torch.zeros(len(indep_fps))
        ]).to(device)

        # Create classifier
        classifier = VerificationClassifier(
            input_dim=target_fp.shape[0],
            hidden_dim=self.hidden_dim
        ).to(device)

        optimizer = torch.optim.AdamW(
            classifier.parameters(),
            lr=self.lr,
            weight_decay=self.weight_decay
        )

        # Training loop with early stopping
        best_loss = float('inf')
        patience = 15
        patience_counter = 0
        best_state = None

        classifier.train()
        for epoch in tqdm(range(300), desc="Training verifier"):
            optimizer.zero_grad()

            outputs = classifier(X).squeeze()
            loss = F.binary_cross_entropy(outputs, y)

            # L2 regularization
            l2_reg = torch.tensor(0.).to(device)
            for param in classifier.parameters():
                l2_reg += torch.norm(param)
            loss += 0.01 * l2_reg

            loss.backward()
            optimizer.step()

            if loss.item() < best_loss:
                best_loss = loss.item()
                best_state = classifier.state_dict()
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                break

        # Load best model
        classifier.load_state_dict(best_state)
        self.classifier = classifier
        return classifier

    def verify(self, classifier, target_model, suspect_model, test_data):
        """Verify model ownership"""
        suspect_fp = self.extract_fingerprint(suspect_model, test_data)

        classifier.eval()
        with torch.no_grad():
            suspect_fp = suspect_fp.unsqueeze(0).to(device)
            score = classifier(suspect_fp.to(device)).item()

        return score > 0.5

def train_model(model, data, epochs=200):
    """Train a GNN model with improved training process"""
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

    best_val_loss = float('inf')
    best_state = None
    patience = 20
    patience_counter = 0

    model.train()
    for epoch in tqdm(range(epochs), desc="Training model"):
        optimizer.zero_grad()

        out = model(data.x, data.edge_index)
        loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])

        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = F.cross_entropy(out[data.val_mask], data.y[data.val_mask])
        model.train()

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    model.load_state_dict(best_state)
    return model

In [ ]:
def evaluate_performance(verifier, target_model, extracted_models, indep_models, test_data):
    """Evaluate verification performance"""
    results = []
    labels = []

    # Test extracted models
    for model in tqdm(extracted_models, desc="Testing extracted models"):
        is_extracted = verifier.verify(
            verifier.classifier, target_model, model, test_data
        )
        results.append(is_extracted)
        labels.append(1)

    # Test independent models
    for model in tqdm(indep_models, desc="Testing independent models"):
        is_extracted = verifier.verify(
            verifier.classifier, target_model, model, test_data
        )
        results.append(is_extracted)
        labels.append(0)

    results = np.array(results)
    labels = np.array(labels)

    metrics = {
        'accuracy': np.mean(results == labels),
        'fpr': np.mean((results == 1) & (labels == 0)),
        'fnr': np.mean((results == 0) & (labels == 1))
    }

    return metrics

def run_experiment(dataset_name='Cora', setting='I'):
    """Run complete verification experiment"""
    print(f"\nRunning experiment on {dataset_name} with Setting {setting}")

    # Load dataset
    dataset = Planetoid(root=f'/tmp/{dataset_name}', name=dataset_name, transform=NormalizeFeatures())
    data = dataset[0].to(device)

    # Set architecture and parameters based on setting
    if setting in ['I', 'II']:
        arch = 'GCN'
    else:
        arch = 'SAGE'

    num_layers = 3 if setting in ['II', 'IV'] else 2

    # Create target model
    target_model = EnhancedGNN(
        in_channels=dataset.num_features,
        hidden_channels=128,
        out_channels=dataset.num_classes,
        num_layers=num_layers,
        dropout=0.5,
        arch=arch
    ).to(device)

    print("Training target model...")
    target_model = train_model(target_model, data)

    # Create and train shadow models
    print("\nTraining shadow models...")
    shadow_models = []
    for i in range(10):
        model = EnhancedGNN(
            in_channels=dataset.num_features,
            hidden_channels=128,
            out_channels=dataset.num_classes,
            num_layers=num_layers,
            dropout=0.5,
            arch=arch
        ).to(device)
        shadow_models.append(train_model(model, data))

    # Create and train independent models
    print("\nTraining independent models...")
    indep_models = []
    architectures = ['GCN', 'GAT', 'SAGE', 'GIN']
    for i in range(10):
        model = EnhancedGNN(
            in_channels=dataset.num_features,
            hidden_channels=128,
            out_channels=dataset.num_classes,
            num_layers=num_layers,
            dropout=0.5,
            arch=architectures[i % len(architectures)]
        ).to(device)
        indep_models.append(train_model(model, data))

    # Initialize and train verifier
    print("\nTraining verifier...")
    #verifier = GN
    # Initialize and train verifier
    print("\nTraining verifier...")
    verifier = GNNVerifier(
        hidden_dim=128,
        mask_ratio=0.1,
        lr=0.001,
        weight_decay=1e-4
    )

    verifier.train_verifier(target_model, shadow_models, indep_models, data)

    # Evaluate performance
    print("\nEvaluating performance...")
    metrics = evaluate_performance(
        verifier,
        target_model,
        shadow_models[:5],  # Use half for testing
        indep_models[:5],   # Use half for testing
        data
    )

    return metrics

In [ ]:
def main():
    """Main execution function"""
    # List of datasets and settings to test
    datasets = ['Cora', 'CiteSeer', 'PubMed']
    settings = ['I', 'II', 'III', 'IV']

    # Store results
    all_results = {}

    # Run experiments
    for dataset in datasets:
        print(f"\n{'='*50}")
        print(f"Running experiments on {dataset} dataset")
        print(f"{'='*50}")

        dataset_results = {}
        for setting in settings:
            # Run experiment
            metrics = run_experiment(dataset, setting)

            # Store results
            dataset_results[setting] = metrics

            # Print results
            print(f"\nResults for Setting {setting}:")
            print(f"Accuracy: {metrics['accuracy']:.4f}")
            print(f"False Positive Rate: {metrics['fpr']:.4f}")
            print(f"False Negative Rate: {metrics['fnr']:.4f}")

        all_results[dataset] = dataset_results

    # Print summary
    print("\n\n" + "="*50)
    print("FINAL RESULTS SUMMARY")
    print("="*50)

    for dataset in datasets:
        print(f"\n{dataset} Dataset:")
        for setting in settings:
            metrics = all_results[dataset][setting]
            print(f"\nSetting {setting}:")
            print(f"Accuracy: {metrics['accuracy']:.4f}")
            print(f"False Positive Rate: {metrics['fpr']:.4f}")
            print(f"False Negative Rate: {metrics['fnr']:.4f}")


In [ ]:
!pip install --upgrade torch-geometric

In [ ]:
if __name__ == "__main__":
    # Ensure CUDA is available
    if torch.cuda.is_available():
        print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
    else:
        print("CUDA is not available. Using CPU")

    # Run main function
    main()

CUDA is available. Using GPU: Tesla T4

Running experiments on Cora dataset

Running experiment on Cora with Setting I
Training target model...


Training model:  12%|█▏        | 24/200 [00:00<00:00, 207.13it/s]



Training shadow models...


Training model:  13%|█▎        | 26/200 [00:00<00:00, 177.32it/s]



Training independent models...


...


Training verifier...

Training verifier...


Training verifier:  50%|████▉     | 149/300 [00:00<00:00, 234.25it/s]



Evaluating performance...


Testing independent models: 100%|██████████| 5/5 [00:00<00:00, 76.77it/s]


Results for Setting IV:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000


FINAL RESULTS SUMMARY

Cora Dataset:

Setting I:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

Setting II:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

Setting III:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

Setting IV:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

CiteSeer Dataset:

Setting I:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

Setting II:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

Setting III:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

Setting IV:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

PubMed Dataset:

Setting I:
Accuracy: 1.0000
False Positive Rate: 0.0000
False Negative Rate: 0.0000

Setting II:
Accuracy: 1.0000
False Positive Rate: 0.0